In [1]:
import numpy as np
import pickle
import tensorflow as tf
import random

In [ ]:
#load the data set
with open('./data/velocity.pickle', 'rb') as handle:
    velocities = pickle.load(handle)

velocities.shape

In [2]:
#Build the discriminator

def convolution2d(input, biases, weights, strides, padding_kind='SAME'):
    input = tf.nn.conv2d(input, weights, [1,strides,strides,1], padding=padding_kind)
    input = tf.nn.bias_add(input, biases)
    input = tf.nn.leaky_relu(input)
    return input

def discriminator(x_image):
    
    #layer1: Convolution
    weights1=tf.Variable(tf.random_normal([12,12,2,2],name='d_Wconv1', stddev=0.01))
    #[filter_height, filter_width, in_channels, out_channels]
    #bias=out_channels
    bias1=tf.Variable(tf.random_normal([2], name='d_Bconv1',stddev=0.01))
    stride1=2
    out1=convolution2d(x_image,bias1,weights1,stride1)
    print(out1.shape)
    #layer2: Convolution
    weights2=tf.Variable(tf.random_normal([6,6,2,4],name='d_Wconv2', stddev=0.01))
    bias2=tf.Variable(tf.random_normal([4],name='d_Bconv2', stddev=0.01))
    stride2=4
    out2=convolution2d(out1,bias2,weights2,stride2)
    print(out2.shape)
    #layer3: Convolution
    weights3=tf.Variable(tf.random_normal([4,4,4,8], name='d_Wconv3',stddev=0.01))
    bias3=tf.Variable(tf.random_normal([8], name='d_Bconv3',stddev=0.01))
    stride3=2
    out3=convolution2d(out2,bias3,weights3,stride3)
    print(out3.shape)
    #layer4: Convolution
    weights4=tf.Variable(tf.random_normal([3,3,8,16],name='d_Wconv4', stddev=0.01))#weights==filters
    bias4=tf.Variable(tf.random_normal([16],name='d_Bconv4', stddev=0.01))
    stride4=2
    out4=convolution2d(out3,bias4,weights4,stride4)
    print(out4.shape)
    #layer5: Fully Connected Layer
    out4 = tf.reshape(out4, shape=[-1, 64 ]) # flatten
    fc_1weights = tf.Variable(tf.random_normal([64, 1], name='d_WFCN1',stddev=0.01))
    fc_1bias   = tf.Variable(tf.random_normal([1],name='d_BFCN1', stddev=0.01))
    fc1 = tf.add(tf.matmul(out4, fc_1weights), fc_1bias)
    fc1 = tf.nn.tanh(fc1)
    
    return fc1

In [11]:
#Build the Generator

def deconvolution2d(input, weights, biases,outputShape, strides, padding_kind='SAME',activation='leaky'):
    # needed for dynamic shape with deconvolution
    dynamicBatchSize = tf.shape(input)[0]
    deconvShape = tf.stack([dynamicBatchSize, outputShape[1], outputShape[2], outputShape[3]])
    input = tf.nn.conv2d_transpose(input, weights, deconvShape, [1,strides,strides,1], padding=padding_kind)
    input = tf.nn.bias_add(input, biases)
    if activation =='leaky':
        input = tf.nn.leaky_relu(input)
    elif activation=='tanh':
        input = tf.nn.tanh(input)
    return input

def generator(noise):
    #layer1: DeConvolution
    weights5=tf.Variable(tf.random_normal([3,3,8,16], name='g_Wdeconv1',stddev=0.01))#weights==filters
    #[filter_height, filter_width, in_channels, out_channels]
    #bias=out_channels
    bias5=tf.Variable(tf.random_normal([8],name='g_Bdeconv1', stddev=0.01))
    stride5=2
    deconv1=deconvolution2d(noise,weights5,bias5,[None, 4,4, 8],stride5)

    #layer2: DeConvolution
    weights6=tf.Variable(tf.random_normal([4,4,4,8],name='g_Wdeconv2', stddev=0.01))#weights==filters
    bias6=tf.Variable(tf.random_normal([4], name='g_Bdeconv2',stddev=0.01))
    stride6=2
    deconv2=deconvolution2d(deconv1,weights6,bias6,[None, 8,8, 4],stride6)

    #layer7: DeConvolution
    weights7=tf.Variable(tf.random_normal([6,6,2,4], name='g_Wdeconv3', stddev=0.01))#weights==filters
    bias7=tf.Variable(tf.random_normal([2], name='g_Bdeconv3', stddev=0.01))
    stride7=4
    deconv3=deconvolution2d(deconv2,weights7,bias7,[None, 32,32, 2],stride7)

    #layer8: DeConvolution
    weights8=tf.Variable(tf.random_normal([12,12,2,2],name='g_Wdeconv4', stddev=0.01))#weights==filters
    bias8=tf.Variable(tf.random_normal([2], name='g_Bdeconv4', stddev=0.01))
    stride8=2
    xOut=deconvolution2d(deconv3,weights8,bias8,[None, 64,64, 2],stride8,activation='tanh')# based on GANHacks
    
    return xOut



In [12]:
#Training
batch_size = 10
sess = tf.Session()
Disc_placeholder = tf.placeholder(tf.float32,shape = [None, 64,64, 2])
Gen_placeholder = tf.placeholder(tf.float32,shape=[None, 2,2, 16])


In [13]:
Disc_real = discriminator(Disc_placeholder) #holds discriminator outputs (unnormalized) for the real images
Gen_real = generator(Gen_placeholder) #Gen_real holds the generated images
Disc_gen = discriminator(Gen_real) #will hold the discriminator output (unnormalized) for generated images


(?, 32, 32, 2)
(?, 8, 8, 4)
(?, 4, 4, 8)
(?, 2, 2, 16)
(?, 32, 32, 2)
(?, 8, 8, 4)
(?, 4, 4, 8)
(?, 2, 2, 16)


In [14]:
#loss functions
g_loss = tf.metrics.mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Disc_gen, labels=tf.ones_like(Disc_gen)))

In [15]:
#loss function for discriminator
d_loss_real = tf.metrics.mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Disc_real, labels=tf.ones_like(Disc_real)))
d_loss_fake = tf.metrics.mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Disc_gen, labels=tf.zeros_like(Disc_gen)))
d_loss = d_loss_real+d_loss_fake

In [16]:
tvars = tf.trainable_variables()

In [17]:
tvars

[<tf.Variable 'Variable:0' shape=(12, 12, 2, 2) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(6, 6, 2, 4) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'Variable_4:0' shape=(4, 4, 4, 8) dtype=float32_ref>,
 <tf.Variable 'Variable_5:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'Variable_6:0' shape=(3, 3, 8, 16) dtype=float32_ref>,
 <tf.Variable 'Variable_7:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'Variable_8:0' shape=(64, 1) dtype=float32_ref>,
 <tf.Variable 'Variable_9:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'Variable_10:0' shape=(3, 3, 8, 16) dtype=float32_ref>,
 <tf.Variable 'Variable_11:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'Variable_12:0' shape=(4, 4, 4, 8) dtype=float32_ref>,
 <tf.Variable 'Variable_13:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'Variable_14:0' shape=(6, 6, 2, 4) dtype=float32_ref>,
 <tf.Variable 'Variable_15:0' sha

In [ ]:
d_vars# = [var for var in tvars if 'd_' in var.name]

In [ ]:
tvars = tf.trainable_variables()
d_vars = [var for var in tvars if 'd_' in var.name]
g_vars = [var for var in tvars if 'g_' in var.name]

In [ ]:
with tf.variable_scope(tf.get_variable_scope(), reuse=False):
    trainerD = tf.train.AdamOptimizer().minimize(d_loss, var_list=d_vars)
    trainerG = tf.train.AdamOptimizer().minimize(g_loss, var_list=g_vars)

In [10]:
###Counting the learnable parameters
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    #print(variable_parameters)
    total_parameters += variable_parameters
print("Total No. Learnable Parameters for the GAN:",total_parameters)


Total No. Learnable Parameters for the GAN: 7790
